# 🎭 LLM-Based Deepfake Reasoning Agent

## Government Cyber-Fraud Prevention Project

### Why Deepfake Detection Matters

Deepfakes have become a critical threat in **digital arrest scams**, where fraudsters impersonate:
- Police officers
- Government officials
- Court judges
- Law enforcement agents

These AI-generated videos are used to:
1. **Intimidate victims** into believing they're under investigation
2. **Extort money** through fake legal threats
3. **Steal personal information** under the guise of official procedures

**This notebook represents a Stage-1 prototype** that uses LLM reasoning to identify suspicious videos before they can be used in scams.

## 🤖 What This Agent Is (Ed Donner Style)

### The Difference: Model-Based vs. Reasoning-Based Detection

**Traditional Model-Based Detection:**
- Requires thousands of labeled deepfake videos
- Needs CNNs, temporal models, face landmark tracking
- Takes weeks/months to train
- High accuracy but slow to deploy

**Our Reasoning-Based Agent (Stage-1):**
- Uses an LLM to **reason** about video characteristics
- No training required
- Can be deployed in hours
- Lower accuracy but provides **immediate value**

### Why Start with a Reasoning Agent?

> *"Start simple, prove value, then scale."* — Ed Donner

This agent:
1. **Validates the concept** quickly
2. **Provides early warnings** while we build better models
3. **Helps us understand** what features matter most
4. **Buys time** to collect training data

Think of it as a **smart triage system** — not perfect, but useful enough to catch obvious fakes and flag suspicious cases for human review.

In [7]:
# Import necessary libraries
import os
import json
from datetime import datetime

# For LLM integration - using Ollama with local Llama 3.2 model
try:
    import ollama
    # Test if Ollama is running and model is available
    try:
        ollama.chat(model='llama3.2:latest', messages=[{'role': 'user', 'content': 'test'}])
        LLM_AVAILABLE = True
        print("✅ Ollama connected successfully with llama3.2:latest model")
    except Exception as e:
        print(f"⚠️ Ollama error: {e}")
        print("   Make sure Ollama is running and llama3.2:latest model is downloaded")
        print("   Run: ollama pull llama3.2:latest")
        LLM_AVAILABLE = False
except ImportError:
    print("⚠️ Ollama library not installed. Install with: pip install ollama")
    print("   Will use mock responses for demo.")
    LLM_AVAILABLE = False

# Optional: OpenCV for video frame extraction (not required for basic demo)
try:
    import cv2
    CV2_AVAILABLE = True
    print("✅ OpenCV available for video analysis")
except ImportError:
    print("⚠️ OpenCV not installed. Install with: pip install opencv-python")
    print("   Will use mock video metadata for demo.")
    CV2_AVAILABLE = False

print("\n✅ Libraries imported successfully")
print(f"📅 Today's date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Ollama connected successfully with llama3.2:latest model
✅ OpenCV available for video analysis

✅ Libraries imported successfully
📅 Today's date: 2026-02-03


## 📊 Video Metadata Extraction

Before we ask the LLM to reason about a video, we need to extract **observable signals** that might indicate manipulation:

### Simple Observable Signals:
1. **Frame count** — Deepfakes often have inconsistent frame rates
2. **Video duration** — Scam videos are typically short (30-60 seconds)
3. **Frame rate** — Unusual frame rates can indicate re-encoding
4. **Resolution** — Low resolution can hide artifacts
5. **Face detection** — Is there a clear face in the video?

**Note:** We're not doing complex analysis here — just gathering basic facts that the LLM can reason about.

In [8]:
def extract_video_metadata(video_path):
    """
    Extract simple metadata from a video file.
    
    This function gathers basic observable signals without deep analysis.
    
    Args:
        video_path: Path to the video file
    
    Returns:
        Dictionary with video metadata
    """
    
    if not CV2_AVAILABLE:
        # Return mock data for demo purposes
        print("⚠️ OpenCV not available, using mock metadata")
        return {
            'file_name': os.path.basename(video_path) if video_path else 'mock_video.mp4',
            'duration_seconds': 45,
            'frame_count': 1350,
            'fps': 30.0,
            'resolution': '1280x720',
            'face_detected': True,
            'file_size_mb': 12.5
        }
    
    # Real video analysis with OpenCV
    try:
        video = cv2.VideoCapture(video_path)
        
        # Extract basic properties
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = video.get(cv2.CAP_PROP_FPS)
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0
        
        # Get file size
        file_size_mb = os.path.getsize(video_path) / (1024 * 1024)
        
        # Simple face detection heuristic (optional)
        # For Stage-1, we'll just assume face is present if video is valid
        face_detected = frame_count > 0
        
        video.release()
        
        return {
            'file_name': os.path.basename(video_path),
            'duration_seconds': round(duration, 2),
            'frame_count': frame_count,
            'fps': round(fps, 2),
            'resolution': f'{width}x{height}',
            'face_detected': face_detected,
            'file_size_mb': round(file_size_mb, 2)
        }
    
    except Exception as e:
        print(f"⚠️ Error extracting metadata: {str(e)}")
        return None


# Example usage
print("📹 Example Video Metadata Extraction:\n")

# Mock example for demonstration
example_metadata = extract_video_metadata("fake_officer_call.mp4")
print(json.dumps(example_metadata, indent=2))

📹 Example Video Metadata Extraction:

{
  "file_name": "fake_officer_call.mp4",
  "duration_seconds": 28.84,
  "frame_count": 721,
  "fps": 25.0,
  "resolution": "640x480",
  "face_detected": true,
  "file_size_mb": 1.46
}


## 🧠 Agent Reasoning Strategy

### How the Agent Works

Our agent doesn't "detect" deepfakes in the traditional sense. Instead, it **reasons** about them like a human analyst would:

#### Step 1: Observe Video Characteristics
- Duration (scam videos are usually 30-90 seconds)
- Quality (low resolution can hide artifacts)
- Frame rate (unusual values suggest re-encoding)

#### Step 2: Receive Human-Readable Cues
- User description: "Claims to be a police officer"
- Context: "Demanding immediate payment"
- Behavior: "Threatening legal action"

#### Step 3: Make a Probabilistic Judgment
The LLM considers:
- **Technical signals** (metadata)
- **Contextual signals** (user description)
- **Known patterns** (common scam tactics)

#### Output:
- **Classification**: Likely Real / Suspicious / Likely Deepfake
- **Confidence**: Low / Medium / High
- **Reasoning**: Plain language explanation

### Why This Works (Even Without Training)

LLMs have been trained on vast amounts of text about:
- Deepfake technology
- Video forensics
- Scam patterns
- Digital fraud tactics

They can apply this **general knowledge** to reason about specific cases, even without seeing the actual video pixels.

In [9]:
def build_deepfake_analysis_prompt(metadata, user_description):
    """
    Construct a structured prompt for the LLM to analyze deepfake likelihood.
    
    Args:
        metadata: Dictionary with video metadata
        user_description: Human-readable description of the video content
    
    Returns:
        Formatted prompt string
    """
    
    prompt = f"""
You are a deepfake detection analyst for a government cyber-fraud prevention unit.

A video has been submitted for analysis. Your task is to assess whether this video is likely a deepfake used in a scam.

VIDEO METADATA:
- File name: {metadata['file_name']}
- Duration: {metadata['duration_seconds']} seconds
- Frame count: {metadata['frame_count']} frames
- Frame rate: {metadata['fps']} fps
- Resolution: {metadata['resolution']}
- Face detected: {metadata['face_detected']}
- File size: {metadata['file_size_mb']} MB

USER DESCRIPTION:
{user_description}

ANALYSIS QUESTIONS:
1. Does this video show signs of manipulation based on the metadata?
2. Are the technical characteristics (duration, resolution, file size) consistent with typical scam videos?
3. Does the user description suggest impersonation of authority figures?
4. Is this likely a deepfake used in a digital arrest scam?

RESPOND IN THE FOLLOWING JSON FORMAT:
{{
    "classification": "Likely Real" | "Suspicious" | "Likely Deepfake",
    "confidence": "Low" | "Medium" | "High",
    "reasoning": "Your detailed explanation in 2-3 sentences",
    "red_flags": ["list of suspicious indicators"],
    "recommendation": "What action should be taken"
}}

Be concise and actionable. Focus on practical indicators that law enforcement can act on.
"""
    
    return prompt


# Example prompt construction
print("📝 Example Deepfake Analysis Prompt:\n")
print("=" * 80)

example_metadata = {
    'file_name': 'suspicious_officer_call.mp4',
    'duration_seconds': 45,
    'frame_count': 1350,
    'fps': 30.0,
    'resolution': '640x480',
    'face_detected': True,
    'file_size_mb': 8.2
}

example_description = """
Video shows a person claiming to be a senior police officer from the Cyber Crime Division.
The person is wearing what appears to be a police uniform and is demanding immediate payment
of ₹50,000 to avoid arrest for alleged money laundering. The caller threatens that failure to
pay will result in a digital arrest warrant being issued within 2 hours.
"""

example_prompt = build_deepfake_analysis_prompt(example_metadata, example_description)
print(example_prompt)
print("=" * 80)

📝 Example Deepfake Analysis Prompt:


You are a deepfake detection analyst for a government cyber-fraud prevention unit.

A video has been submitted for analysis. Your task is to assess whether this video is likely a deepfake used in a scam.

VIDEO METADATA:
- File name: suspicious_officer_call.mp4
- Duration: 45 seconds
- Frame count: 1350 frames
- Frame rate: 30.0 fps
- Resolution: 640x480
- Face detected: True
- File size: 8.2 MB

USER DESCRIPTION:

Video shows a person claiming to be a senior police officer from the Cyber Crime Division.
The person is wearing what appears to be a police uniform and is demanding immediate payment
of ₹50,000 to avoid arrest for alleged money laundering. The caller threatens that failure to
pay will result in a digital arrest warrant being issued within 2 hours.


ANALYSIS QUESTIONS:
1. Does this video show signs of manipulation based on the metadata?
2. Are the technical characteristics (duration, resolution, file size) consistent with typical scam v

## 🎯 LLM Deepfake Judgment

This is the core of our reasoning agent. The LLM acts as an **intelligent analyst** that:

1. **Examines** the video metadata
2. **Considers** the context and user description
3. **Applies** knowledge about deepfakes and scams
4. **Provides** a structured judgment with reasoning

### Output Format:
- **Classification**: Likely Real / Suspicious / Likely Deepfake
- **Confidence**: Low / Medium / High
- **Reasoning**: Why the agent made this decision
- **Red Flags**: Specific suspicious indicators
- **Recommendation**: What action to take

In [10]:
def analyze_deepfake_with_llm(metadata, user_description):
    """
    Use LLM to analyze whether a video is likely a deepfake.
    
    Args:
        metadata: Dictionary with video metadata
        user_description: Human-readable description of the video
    
    Returns:
        Dictionary with analysis results
    """
    
    if not LLM_AVAILABLE:
        # Return mock analysis for demo
        print("⚠️ LLM not available, using mock analysis")
        return {
            'classification': 'Suspicious',
            'confidence': 'Medium',
            'reasoning': 'Mock analysis: Low resolution and short duration are common in scam videos. User description mentions authority impersonation and payment demands, which are red flags for digital arrest scams.',
            'red_flags': [
                'Low resolution (640x480) can hide deepfake artifacts',
                'Short duration (45 seconds) typical of scam videos',
                'Claims to be police officer demanding immediate payment',
                'Threatens arrest within unrealistic timeframe (2 hours)'
            ],
            'recommendation': 'Flag for human review. Advise victim not to make any payments. Report to cybercrime authorities.'
        }
    
    # Build the analysis prompt
    prompt = build_deepfake_analysis_prompt(metadata, user_description)
    
    try:
        # Use Ollama with local Llama 3.2 model
        response = ollama.chat(
            model='llama3.2:latest',
            messages=[
                {
                    'role': 'system',
                    'content': 'You are a deepfake detection analyst. Provide structured, actionable analysis in JSON format.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ],
            options={
                'temperature': 0.3,  # Lower temperature for consistent analysis
                'num_predict': 300   # Limit response length
            }
        )
        
        # Extract the response
        llm_response = response['message']['content'].strip()
        
        # Try to parse JSON response
        try:
            # Remove markdown code blocks if present
            if '```json' in llm_response:
                llm_response = llm_response.split('```json')[1].split('```')[0].strip()
            elif '```' in llm_response:
                llm_response = llm_response.split('```')[1].split('```')[0].strip()
            
            analysis = json.loads(llm_response)
            return analysis
        
        except json.JSONDecodeError:
            # If JSON parsing fails, return raw response
            print("⚠️ Could not parse JSON response, returning raw analysis")
            return {
                'classification': 'Unknown',
                'confidence': 'Low',
                'reasoning': llm_response,
                'red_flags': [],
                'recommendation': 'Manual review required'
            }
    
    except Exception as e:
        print(f"⚠️ LLM error: {str(e)}")
        return {
            'classification': 'Error',
            'confidence': 'N/A',
            'reasoning': f'Analysis failed: {str(e)}',
            'red_flags': [],
            'recommendation': 'Manual review required'
        }


def print_analysis_report(video_name, metadata, user_description, analysis):
    """
    Print a clean, formatted analysis report.
    """
    print("\n" + "=" * 80)
    print(f"🎭 DEEPFAKE ANALYSIS REPORT")
    print("=" * 80)
    print(f"\n📹 Video: {video_name}")
    print(f"⏱️  Duration: {metadata['duration_seconds']}s | Resolution: {metadata['resolution']} | FPS: {metadata['fps']}")
    print(f"\n📝 Description:\n{user_description.strip()}")
    print("\n" + "-" * 80)
    print(f"\n🎯 CLASSIFICATION: {analysis['classification']}")
    print(f"📊 CONFIDENCE: {analysis['confidence']}")
    print(f"\n💡 REASONING:\n{analysis['reasoning']}")
    
    if analysis['red_flags']:
        print(f"\n🚩 RED FLAGS:")
        for flag in analysis['red_flags']:
            print(f"   • {flag}")
    
    print(f"\n✅ RECOMMENDATION:\n{analysis['recommendation']}")
    print("\n" + "=" * 80)

## 🎬 Analyze Your Video

Now let's analyze a real video file. You need to provide:

1. **Video file path** - Path to the video file you want to analyze
2. **Video description** - What you observed in the video (claims, behavior, context)

The agent will:
- Extract metadata from the video
- Analyze it using the LLM
- Provide a classification with reasoning

In [ ]:
# ========================================
# USER INPUT: Specify your video details
# ========================================

# Path to your video file (update this with your actual video path)
VIDEO_PATH = "fake_officer_call.mp4"  # Change this to your video file path

# Description of what you observed in the video
VIDEO_DESCRIPTION = """
Video shows a person in a police uniform claiming to be a senior officer.
The person is demanding immediate payment and threatening arrest.
Background appears to be a police station but lighting seems artificial.
"""  # Update this with your observations

print("📋 Video Analysis Configuration:")
print(f"   Video Path: {VIDEO_PATH}")
print(f"   Description: {VIDEO_DESCRIPTION.strip()[:100]}...")
print("\n" + "="*80)


################################################################################
# SCENARIO 1: Suspicious Fake Officer Video
################################################################################

🎭 DEEPFAKE ANALYSIS REPORT

📹 Video: police_officer_urgent_call.mp4
⏱️  Duration: 38s | Resolution: 640x480 | FPS: 30.0

📝 Description:
Video shows a person in a police uniform claiming to be "DCP Rajesh Kumar from Cyber Crime Cell".
The person states that the recipient's Aadhaar card has been used in a money laundering case
involving ₹2.5 crores. Demands immediate payment of ₹75,000 as "security deposit" to avoid arrest.
Claims a digital arrest warrant will be issued in 90 minutes if payment is not made.
Background shows what appears to be a police station, but the lighting seems artificial.

--------------------------------------------------------------------------------

🎯 CLASSIFICATION: Suspicious
📊 CONFIDENCE: Medium

💡 REASONING:
The video shows signs of artificial lighting,

In [ ]:
# ========================================
# STEP 1: Extract Video Metadata
# ========================================

print("\n🔍 STEP 1: Extracting video metadata...\n")

video_metadata = extract_video_metadata(VIDEO_PATH)

if video_metadata:
    print("✅ Metadata extracted successfully:")
    print(json.dumps(video_metadata, indent=2))
else:
    print("❌ Failed to extract metadata. Please check the video path.")
    print(f"   Attempted path: {VIDEO_PATH}")


################################################################################
# SCENARIO 2: Normal Video Call
################################################################################

🎭 DEEPFAKE ANALYSIS REPORT

📹 Video: family_video_call.mp4
⏱️  Duration: 180s | Resolution: 1920x1080 | FPS: 30.0

📝 Description:
Video call recording with a family member discussing weekend plans.
Natural conversation with normal pauses, laughter, and casual topics.
Background shows a home environment with visible furniture and decorations.
No requests for money or urgent actions. Person appears relaxed and natural.

--------------------------------------------------------------------------------

🎯 CLASSIFICATION: Likely Real
📊 CONFIDENCE: Medium

💡 REASONING:
The video's natural conversation, relaxed tone, and lack of urgency or requests for money suggest a genuine family call. The metadata is consistent with typical personal video calls, and there are no obvious signs of manipulation. Howe

In [ ]:
# ========================================
# STEP 2: Analyze with LLM
# ========================================

if video_metadata:
    print("\n🤖 STEP 2: Analyzing video with LLM...\n")
    
    # Perform deepfake analysis
    analysis_result = analyze_deepfake_with_llm(video_metadata, VIDEO_DESCRIPTION)
    
    # Print the detailed report
    print_analysis_report(video_metadata['file_name'], video_metadata, VIDEO_DESCRIPTION, analysis_result)
    
    # Summary
    print("\n" + "="*80)
    print("📊 ANALYSIS COMPLETE")
    print("="*80)
    print(f"\n🎯 Final Classification: {analysis_result['classification']}")
    print(f"📊 Confidence Level: {analysis_result['confidence']}")
    print(f"\n⚠️ Remember: This is a Stage-1 prototype. Always verify with human review.")
else:
    print("\n❌ Cannot proceed with analysis - metadata extraction failed.")
    print("   Please check your video file path and try again.")

## ⚠️ IMPORTANT LIMITATIONS

### This is NOT Forensic Detection

**What this agent CANNOT do:**
- ❌ Detect sophisticated deepfakes with high accuracy
- ❌ Analyze actual video frames for artifacts
- ❌ Verify facial landmarks or micro-expressions
- ❌ Check audio-visual synchronization
- ❌ Compare against known identity databases
- ❌ Provide forensic-grade evidence for court

**What this agent CAN do:**
- ✅ Provide early warnings based on contextual analysis
- ✅ Flag obviously suspicious videos for human review
- ✅ Identify common scam patterns and tactics
- ✅ Triage cases to prioritize investigation resources
- ✅ Educate users about deepfake red flags

### Use Cases:

**✅ Good for:**
- Initial screening of reported scam videos
- Public awareness and education
- Rapid triage in high-volume scenarios
- Proof-of-concept for stakeholder buy-in

**❌ NOT suitable for:**
- Legal evidence in court cases
- High-stakes security decisions
- Automated blocking without human review
- Cases requiring forensic certainty

### Accuracy Expectations:

This Stage-1 agent is expected to:
- Catch **obvious scams** with high confidence
- Flag **suspicious cases** for human review
- Miss **sophisticated deepfakes** that require technical analysis

**Always require human verification before taking action.**

## 🚀 Future Enhancements (Stage-2 and Beyond)

This Stage-1 prototype proves the concept. Here's how we evolve it into a production system:

### **Stage-2: Add Technical Detection**
- **CNN-based face analysis**: Train models to detect facial artifacts
- **Temporal inconsistency detection**: Analyze frame-to-frame changes
- **Face landmark tracking**: Detect unnatural facial movements
- **Audio-visual sync analysis**: Check if lip movements match speech
- **Blink pattern analysis**: Deepfakes often have abnormal blinking

### **Stage-3: Identity Verification**
- **Official identity database**: Compare against verified government officer photos
- **Badge/uniform verification**: Check if uniform details are authentic
- **Voice biometrics**: Analyze voice patterns for authenticity
- **Background verification**: Validate that office environment is genuine

### **Stage-4: Multi-Modal Analysis**
- **Combine video + audio + metadata**: Holistic analysis
- **Behavioral analysis**: Detect scripted vs. natural speech patterns
- **Context verification**: Cross-reference with actual police procedures
- **Network analysis**: Track if video appears in multiple scam reports

### **Stage-5: Production Deployment**
- **Real-time API**: Integrate with reporting platforms
- **Mobile app integration**: Allow citizens to verify videos instantly
- **Dashboard for analysts**: Centralized review and case management
- **Automated alerts**: Notify authorities of high-confidence deepfakes
- **Feedback loop**: Improve models based on analyst corrections

### **Advanced Features**
- **Generative adversarial testing**: Test against latest deepfake techniques
- **Blockchain verification**: Timestamp and verify authentic government videos
- **Public education**: Generate reports explaining why a video is suspicious
- **Cross-agency collaboration**: Share intelligence with other law enforcement units

---

### **Key Principle (Ed Donner Style)**

> *"This notebook proves we can provide value TODAY while building something better for TOMORROW."*

We're not waiting for perfect technology. We're deploying useful intelligence NOW and improving iteratively.

---

## 📚 Resources

**Learn More:**
- [Deepfake Detection Research](https://arxiv.org/abs/2004.11138)
- [Digital Arrest Scam Awareness](https://cybercrime.gov.in)
- [Ed Donner's Agentic AI Course](https://www.udemy.com/course/agentic-ai)

**Report Scams:**
- National Cybercrime Helpline: 1930
- Report online: https://cybercrime.gov.in

---

**Questions? Feedback?**  
This is a living prototype. Test it, improve it, deploy it. That's how we protect citizens from digital fraud. 🛡️